# Multinomial Logistic Regression
# 
## 2018-09-26
# 
## *Qin CAO*
# 
Logistic regression is a widely-used model when the response is categorical. If there are two possible outcomes, we use the binomial distribution, else we use the multinomial.
## Binomial Logistic Regression
For the binomial model, suppose the response variable takes value in $\mathcal{G}=\{1,2\}$. Denote $y_i = I(g_i=1)$. 

We model
$$\mbox{Pr}(G=2|X=x)=\frac{e^{\beta_0+\beta^Tx}}{1+e^{\beta_0+\beta^Tx}}, [1]$$
and thus $$\mbox{Pr}(G=1|X=x)=1-\frac{e^{\beta_0+\beta^Tx}}{1+e^{\beta_0+\beta^Tx}}, [2]$$
which can be written in the following form
$$\log\frac{\mbox{Pr}(G=2|X=x)}{\mbox{Pr}(G=1|X=x)}=\beta_0+\beta^Tx, [3]$$
the so-called “logistic” or log-odds transformation. From the formula, we can see that it is very similar to linear regression except that now we are not modeling reponse directly but rather modeling log-odds linearly. We should note we model the "relative probability" here.

The objective function for the penalized logistic regression uses the negative binomial log-likelihood, and is
$$\min_{(\beta_0, \beta) \in \mathbb{R}^{p+1}} -\left[\frac{1}{N} \sum_{i=1}^N y_i \cdot (\beta_0 + x_i^T \beta) - \log (1+e^{(\beta_0+x_i^T \beta)})\right] + \lambda \big[ (1-\alpha)||\beta||_2^2/2 + \alpha||\beta||_1\big]. [4]$$
Logistic regression is often plagued with degeneracies when $p>N$ and exhibits wild behavior even when $N$ is close to $p$; the elastic-net penalty alleviates these issues, and regularizes and selects variables as well.

## Multinomial Logistic Regression
For the multinomial model, suppose the response variable has $K$ levels ${\cal G}=\{1,2,\ldots,K\}$. Here we model
$$\mbox{Pr}(G=k|X=x)=\frac{e^{\beta_{k}+\beta_k^Tx}}{\sum_{\ell=1}^Ke^{\beta_{0\ell}+\beta_\ell^Tx}}. [5]$$

Let $Y$ be the $N \times K$ indicator response matrix, with elements $y_{i\ell} = I(g_i=\ell)$. Then the elastic-net penalized negative log-likelihood function becomes
$$\ell(\{\beta_{0k},\beta_{k}\}_1^K) = -\left[\frac{1}{N} \sum_{i=1}^N \Big(\sum_{k=1}^Ky_{il} (\beta_{0k} + x_i^T \beta_k)- \log \big(\sum_{k=1}^K e^{\beta_{0k}+x_i^T \beta_k}\big)\Big)\right] +\lambda \left[ (1-\alpha)||\beta||_F^2/2 + \alpha\sum_{j=1}^p||\beta_j||_q\right]. [6]$$

$\beta$ is a $p\times K$ matrix of coefficients. $\beta_k$ refers to the kth column (for outcome category k), and $\beta_j$ the jth row (vector of K coefficients for variable j). The last penalty term is $||\beta_j||_q$, which denotes lasso penalty or grouped-lasso penalty. From the formula, we can get an intuitive feeling that the prediction to a certain class is not only related to the $\beta$ value of the corresponding class but also related to the $\beta$ values of other classes. 

Now we give an example to illustrate how to use multinomial logistic regression. 
> Entering high school students make program choices among general program, vocational program and academic program. Their choice might be modeled using their writing score and their social economic status.

We first read in the data.


In [66]:
require(foreign)
require(nnet)
require(ggplot2)
require(reshape2)
ml <- read.dta("https://stats.idre.ucla.edu/stat/data/hsbdemo.dta")
head(ml,20)

id,female,ses,schtyp,prog,read,write,math,science,socst,honors,awards,cid
45,female,low,public,vocation,34,35,41,29,26,not enrolled,0,1
108,male,middle,public,general,34,33,41,36,36,not enrolled,0,1
15,male,high,public,vocation,39,39,44,26,42,not enrolled,0,1
67,male,low,public,vocation,37,37,42,33,32,not enrolled,0,1
153,male,middle,public,vocation,39,31,40,39,51,not enrolled,0,1
51,female,high,public,general,42,36,42,31,39,not enrolled,0,1
164,male,middle,public,vocation,31,36,46,39,46,not enrolled,0,1
133,male,middle,public,vocation,50,31,40,34,31,not enrolled,0,1
2,female,middle,public,vocation,39,41,33,42,41,not enrolled,0,1
53,male,middle,public,vocation,34,37,46,39,31,not enrolled,0,1


The data set contains variables on 200 students. The outcome variable is prog, program type. The predictor variables are social economic status, ses, a three-level categorical variable and writing score, write, a continuous variable. 

We first need to define a baseline in order to compute the log-odds. From binomial logistic regression model, we actually defined $\mbox{G=1}$ to be the baseline. The absolute probability of $\mbox{G=2}$ is calculated based on the "relative probability" over the probability of $\mbox{G=1}$ (Formulas $[1] [2] [3]$). Here we define "academic" to be the baseline. 

In [67]:
ml$prog2 <- relevel(ml$prog, ref = "academic")
print(ml$prog2)

  [1] vocation general  vocation vocation vocation general  vocation vocation
  [9] vocation vocation vocation academic vocation vocation vocation general 
 [17] general  vocation academic vocation general  vocation vocation vocation
 [25] academic academic general  general  academic academic general  vocation
 [33] academic academic vocation vocation vocation academic general  academic
 [41] general  academic academic vocation academic vocation vocation general 
 [49] vocation academic academic vocation general  academic academic general 
 [57] academic general  vocation general  vocation academic academic vocation
 [65] vocation vocation general  academic academic general  academic academic
 [73] academic general  vocation general  vocation general  general  academic
 [81] vocation academic academic general  vocation academic general  general 
 [89] general  vocation vocation general  vocation academic vocation general 
 [97] academic vocation vocation general  academic vocation voca

Since ses is actually a categorical variable, we need to convert it into dummy variables as the input to the multinomial logistic regression model. Fortunately, nnet will help us to do it automatically. We also manually create the dummy variables to understand the transformation.

In [68]:
dummies = model.matrix(~ml$ses)
head(ml$ses)
head(dummies)

[1] low    middle high   low    middle high  
Levels: low middle high

,(Intercept),ml$sesmiddle,ml$seshigh
1,1,0,0
2,1,1,0
3,1,0,1
4,1,0,0
5,1,1,0
6,1,0,1


So we can see that actually "low" has been tranformed to (0,0), "middle" has been transformed to (1,0) and "high" has been transformed to (0,1). We will see later that Intercept=1 is for computational convenience. Now we build the multinomial model.  

In [69]:
test <- multinom(prog2 ~ ses + write, data = ml)
summary(test)

# weights:  15 (8 variable)
initial  value 219.722458 
iter  10 value 179.982880
final  value 179.981726 
converged


Call:
multinom(formula = prog2 ~ ses + write, data = ml)

Coefficients:
         (Intercept)  sesmiddle    seshigh      write
general     2.852198 -0.5332810 -1.1628226 -0.0579287
vocation    5.218260  0.2913859 -0.9826649 -0.1136037

Std. Errors:
         (Intercept) sesmiddle   seshigh      write
general     1.166441 0.4437323 0.5142196 0.02141097
vocation    1.163552 0.4763739 0.5955665 0.02221996

Residual Deviance: 359.9635 
AIC: 375.9635 

- We first see that some output is generated by running the model, even though we are assigning the model to a new R object. This model-running output includes some iteration history and includes the final negative log-likelihood 179.981726. This value multiplied by two is then seen in the model summary as the Residual Deviance and it can be used in comparisons of nested models.
- The model summary output has a block of coefficients and a block of standard errors. Each of these blocks has one row of values corresponding to a model equation. Focusing on the block of coefficients, we can look at the first row comparing prog = "general" to our baseline prog = "academic" and the second row comparing prog = "vocation" to our baseline prog = "academic". If we consider our coefficients from the first row to be $\beta_1$ and our coefficients from the second row to be $\beta_2$, we can write our model equations:
$$ln\left(\frac{Pr(prog=general)}{Pr(prog=academic)}\right) = \beta_{10} + \beta_{11}(ses=2) + \beta_{12}(ses=3) + \beta_{13}write [7]$$
$$ln\left(\frac{Pr(prog=vocation)}{Pr(prog=academic)}\right) = \beta_{20} + \beta_{21}(ses=2) + \beta_{22}(ses=3) + \beta_{23}write [8]$$
- A one-unit increase in the variable write is associated with the decrease in the log odds of being in general program vs. academic program in the amount of .058 $\beta_{13}$.
- A one-unit increase in the variable write is associated with the decrease in the log odds of being in vocation program vs. academic program. in the amount of .1136 $\beta_{23}$.
- The log odds of being in general program vs. in academic program will decrease by 1.163 if moving from ses="low" to ses="high"$\beta_{12}$.
- The log odds of being in general program vs. in academic program will decrease by 0.533 if moving from ses="low"to ses="middle"$\beta_{11}$, although this coefficient is not significant.
- The log odds of being in vocation program vs. in academic program will decrease by 0.983 if moving from ses="low" to ses="high"$\beta_{22}$.
- The log odds of being in vocation program vs. in academic program will increase by 0.291 if moving from ses="low" to ses="middle"$\beta_{21}$, although this coefficient is not signficant.

We use function "fitted" to apply the learnt multinomial logistic regression model to the original data set. 

In [70]:
head(pp <- fitted(test))

,academic,general,vocation
1,0.1482764,0.3382454,0.5134781
2,0.1202017,0.1806283,0.6991700
3,0.4186747,0.2368082,0.3445171
4,0.1726885,0.3508384,0.4764731
5,0.1001231,0.1689374,0.7309395
6,0.3533566,0.2377976,0.4088458


Given the formula $[5]$ and the learnt parameters (summary(test)), we can compute the predictions manually and double check the results with the output from the function "fitted". 

In [71]:
#Combine ses dummy variables and write
combined <- cbind.data.frame(dummies,ml$write)
combinedm <- as.matrix(combined)
#Get the parameters from the learnt model
coef <- t(coef(test))
coefm <- as.matrix(coef)
#view combined varaibles and parameters
head(combinedm)
print(coefm)

,(Intercept),ml$sesmiddle,ml$seshigh,ml$write
1,1,0,0,35
2,1,1,0,33
3,1,0,1,39
4,1,0,0,37
5,1,1,0,31
6,1,0,1,36


               general   vocation
(Intercept)  2.8521980  5.2182597
sesmiddle   -0.5332810  0.2913859
seshigh     -1.1628226 -0.9826649
write       -0.0579287 -0.1136037


We then times(matrix multiply) "combinedm" with "coefm" to compute the log-odds based on formulas $[7] [8]$. 
After getting log-odds, we take $exp$ and obtain odds. 

In [72]:
relative <- exp(combinedm %*% coefm)
head(relative)

,general,vocation
1,2.2811814,3.4629786
2,1.5027096,5.8166379
3,0.5656137,0.8228752
4,2.0316252,2.7591471
5,1.6872960,7.3004054
6,0.6729678,1.1570347


Now we get the odds("relative probability"), which are namely $\frac{Pr(prog=general)}{Pr(prog=academic)}$ and $\frac{Pr(prog=vocation)}{Pr(prog=academic)}$ respectively. For example, for instance 1, the odds is 2.281, which means that $\frac{Pr(prog=general)}{Pr(prog=academic)}=2.281$. So with all the "relative probabilities", we can compute the absolute probabilities. 

In [73]:
#compute the absolute probability of academic
academic <- 1/(relative[,1]+relative[,2]+1)
#compute the absolute probability of general
general <- academic*as.data.frame(relative)$general
#compute the absolute probability of vocation
vocation <- academic*as.data.frame(relative)$vocation
#our manually calculated result
out <- cbind.data.frame(academic,general,vocation)
print("Our manually calculated result:")
head(out)
#result calculated from the internal function of the package
print("Result calculated from the package:")
head(pp)

[1] "Our manually calculated result:"


academic,general,vocation
0.1482764,0.3382454,0.5134781
0.1202017,0.1806283,0.6991700
0.4186747,0.2368082,0.3445171
0.1726885,0.3508384,0.4764731
0.1001231,0.1689374,0.7309395
0.3533566,0.2377976,0.4088458


[1] "Result calculated from the package:"


,academic,general,vocation
1,0.1482764,0.3382454,0.5134781
2,0.1202017,0.1806283,0.6991700
3,0.4186747,0.2368082,0.3445171
4,0.1726885,0.3508384,0.4764731
5,0.1001231,0.1689374,0.7309395
6,0.3533566,0.2377976,0.4088458


We get correct result! 

To summarize, the multinomial logistic regression model learns the $\beta$ parameters for the log-odds relative to the baseline class. We need to convert the "relative probability" into absolute probability. Going back to the binominal logistic regression model, the negative class is actually set as the baseline. 
## References
* [Glmnet Vignette](https://web.stanford.edu/~hastie/glmnet/glmnet_alpha.html) 
> The basic definitions and formulas of binomial and multinomial logistic regression are from here. The explanations of binomial logistic regression are from here. 
* [UCLA course](https://stats.idre.ucla.edu/r/dae/multinomial-logistic-regression/)
> The example, part of the R source codes and part of the explanations of multinomial logistic regression are from here. 
* [Princeton wws509](http://data.princeton.edu/wws509/stata/mlogit.html)
> Some ideas and thoughts are from here. 

## Notes and Thoughts
Formula  $[6]$ is from [Glmnet Vignette](https://web.stanford.edu/~hastie/glmnet/glmnet_alpha.html), which is actually not the standard loss function for multinomial logistic regression. In binomial case, we have 2 classes and 2-1=1 set of $\beta$ parameters. In multinomial case, we have $K$ classes and $K-1$ sets of $\beta$ parameters. The reason is that we always have a baseline class, and the absolute probabilities across all the other class and the baseline class should sum up to 1. Known the predicted absolute probabilities of the $K-1$ classes, we can automatically know the absolute probability of baseline class. Formula $[6]$ instead has $K$ sets of $\beta$ parameters, which are dealed by a different solving algorithm in R package glmnet. But the property of summming up to 1 is conserved. 